# The impact of rookie QBs on the win probability

As we are now in the second half of the 2020 NFL season, I thought it would be interesting to compare the production of the rookie quarterbacks, i.e. Joe Burrow for the Cincinnati Bengals and Tua Tagovailoa for the Miami Dolphins. What interested me specifically was the impact of their passing game on the win probability, and if it was possible to break it down for each type of pass (deep left, middle right, etc.) 

Before diving into the data, I have to thank [@mnpykings](https://twitter.com/mnpykings) for his incredible [tutorial on the nflfastR with Python](https://github.com/maxbolger/nflfastR-Python-Tutorial/blob/main/nflfastR_python_tutorial.ipynb) and [Hajime Alabanza](https://medium.com/@hajimealabanza?source=post_page-----fe9bb2f37116--------------------------------]) for his fascinating [article](https://medium.com/@hajimealabanza/did-the-new-orleans-saints-add-the-missing-piece-to-their-offense-fe9bb2f37116) on the Saints and his use of the WPA. As he wrote, WPA "is a metric that measures how much more likely a team is to win a game after a given play." In other words, each action impacts the probabilities to win the game, either positively or negatively. This impact is quantified and recorded as "WPA", "Win Probability Added".

## 1 : Import of the librairies and data

Let's begin by importing the different librairies needed:
- numpy and pandas for the cleaning and the handling of the data
- plotly for the interactive visualization

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

Now we download the most recent play-by-play results of the NFL into a dataframe simply called "data". By the time I wrote this article, one game was left to play for week 10 but it doesn't have an impact on this analysis.

In [8]:
# Single season
YEAR = 2020
data = pd.read_csv(
    'https://github.com/guga31bb/nflfastR-data/blob/master/data/play_by_play_' \
    + str(YEAR) + '.csv.gz?raw=True',compression='gzip', low_memory=False
   )

## 2. Cleaning and modification of the data

The table contains 2 columns defining the type of the pass:
- pass_length : short or deep
- pass_location : left, middle, or right

We'll create a new column called "pass_type" by concatenating these two.

Next, we'll calculate a new column called "wpa_pct" by multiplying the current wpa by 100.

In [11]:
data['pass_type'] = data['pass_length'] + ' - ' + data['pass_location']
data['wpa_pct'] = (100*df['wpa'])

Let's remove the data we do not want for our analysis, like kneel downs, field goals, penalties, etc, and assign this new dataframe to "df".

In [13]:
df = data[
    (data.play_type.isin(['no_play','pass','run'])) &
    (data.sack==0) &      
    (data.two_point_attempt==0) &
    (data['epa'].isna()==False)
    ]

Let's take a moment to state the obvious: it is correct that the results we'll get are biased. Indeed, Burrow has played the entirety of the season (i.e. 9 matches + the bye week) while Tua has officially played only 4 matches, and for his first match, he only threw 2 passes as he entered late in the game and only played 5 snaps. We accept this bias as our objective is to get a snap shot of their season so far.

In order to do this, we'll take their data into a new dataframe called "rookies_df" and calculate their basic stats:

In [15]:
rookies_df = df[df.passer_player_name.isin(['T.Tagovailoa','J.Burrow'])]

rookies_df.groupby('passer_player_name').agg(
    games_played=('game_id', 'nunique'),
    pass_attempts=('pass_attempt', 'sum'),
    complete_passes=('complete_pass', 'sum'),
    total_yards_gained=('yards_gained','sum'),
    TD=('pass_touchdown','sum'),
    interceptions=('interception','sum')
)

,games_played,pass_attempts,complete_passes,total_yards_gained,TD,interceptions
passer_player_name,,,,,,
J.Burrow,9,367.0,242.0,2485.0,12.0,5.0
T.Tagovailoa,4,76.0,49.0,519.0,5.0,0.0


## 3: Focus on the wpa

Now we can focus on the wpa stat. One way to analyze them is to group them in a single table:

In [18]:
qb_grouped = rookies_df.groupby(['passer_player_name','pass_type']).agg({'wpa_pct':'mean'})
qb_grouped

wpa_pct
passer_player_name pass_type               
J.Burrow           deep - left    -0.202351
                   deep - middle  -1.698436
                   deep - right    0.378192
                   short - left    0.456855
                   short - middle  0.879427
                   short - right   1.448731
T.Tagovailoa       deep - left     0.608190
                   deep - middle   3.416979
                   deep - right    0.970340
                   short - left    0.551655
                   short - middle  1.195688
                   short - right   0.481609

We have the results, but it's clearly not the best way to compare them. What we are going to do is use an interactive heatmap to represent the field and put the data in a way that is more interesting to the eye.

In order to achieve this, we'll follow several steps:
- Isolate the stats of a QB
- Separate the results for the deep passes and for the sort passes
- Pass the results into two lists
- Pass each list as the source of our visualization
- Customize some parameters in order to make it more interesting
- Repeat the exact same steps for the other QB

In [32]:
#Steps for Joe Burrow
Burrow_WPA = qb_grouped.loc['J.Burrow']
Burrow_deep = Burrow_WPA.loc[['deep - left','deep - middle','deep - right']]
Burrow_deeplist = Burrow_deep.wpa_pct.tolist()
Burrow_short = Burrow_WPA.loc[['short - left','short - middle','short - right']]
Burrow_shortlist = Burrow_short.wpa_pct.tolist()
Burrow_source=[Burrow_deeplist,Burrow_shortlist]
Burrow_fig = px.imshow(Burrow_source, color_continuous_scale ='RdYlGn', zmin=-1.7, zmax=3.5,
                labels=dict(x="Pass location", y="Pass length", color="Average WPA"),
                title='Joe Burrows WPA per pass type',
                x=['Left', 'Middle', 'Right'],
                y=['Deep', 'Short']
               )

#Steps for Tua Tagovailoa
Tagovailoa_WPA = qb_grouped.loc['T.Tagovailoa']
Tagovailoa_deep = Tagovailoa_WPA.loc[['deep - left','deep - middle','deep - right']]
Tagovailoa_deeplist = Tagovailoa_deep.wpa_pct.tolist()
Tagovailoa_short = Tagovailoa_WPA.loc[['short - left','short - middle','short - right']]
Tagovailoa_shortlist = Tagovailoa_short.wpa_pct.tolist()
Tagovailoa_source=[Tagovailoa_deeplist,Tagovailoa_shortlist]
Tagovailoa_fig = px.imshow(Tagovailoa_source, color_continuous_scale ='RdYlGn', zmin=-1.7, zmax=3.5,
                labels=dict(x="Pass location", y="Pass length", color="Average WPA"),
                title='Tua Tagovailoa WPA per pass type',
                x=['Left', 'Middle', 'Right'],
                y=['Deep', 'Short']
               )

Now that our data is ready, we call the two figures we have created:

In [33]:
Burrow_fig.show()
Tagovailoa_fig.show()

As we can see while hovering the mouse above this interactive chart, Joe Burrows is at ease with the short right passes, as they add in average 1.4 points of win probability, but he seems to struggle a lot with the middle deep pass, as they have an average wpa of -1.7. 

On the other hand, this is precisely where Tua is the most impactful : his deep middle passes add on average 3.4 point of win probability!

Sure, as I said earlier, this is a biaised comparison as Burrow has played more than twice the number of games of Tua, and the sample size is realtively small, but this provides some interesting data for the end of this season!